In [1]:
import json

with open('chat_gpt_5.json', 'r') as f:
    chat_gpt_data = json.load(f)

with open('claude_sonnet_4.json', 'r') as f:
    claude_sonnet_data = json.load(f)

with open('gemini_2_5_falsh.json', 'r') as f:
    gemini_flash_data = json.load(f)

print("Chat GPT Data Structure:")
print(type(chat_gpt_data))
print("\nClaude Sonnet Data Structure:")
print(type(claude_sonnet_data))
print("\nGemini Flash Data Structure:")
print(type(gemini_flash_data))


Chat GPT Data Structure:
<class 'list'>

Claude Sonnet Data Structure:
<class 'list'>

Gemini Flash Data Structure:
<class 'list'>


In [2]:
chat_gpt_data[0]['caption']['en']

'Nested gifts, endless surprises.'

In [3]:
chat_gpt_data[0]['description']['en']

'The cartoon shows a set of matryoshka dolls opening gifts. The child doll looks disappointed as the present is just another smaller gift box, while the parents smile behind her.'

In [4]:
gpt_caption = [img['caption']['en'] for img in chat_gpt_data]
gpt_description = [img['description']['en'] for img in chat_gpt_data]

claude_caption = [img['caption']['en'] for img in claude_sonnet_data]
claude_description = [img['description']['en'] for img in claude_sonnet_data]

gemini_caption = [img['caption']['en'] for img in gemini_flash_data]
gemini_description = [img['description']['en'] for img in gemini_flash_data]

In [5]:
print(len(gpt_caption))
print(len(claude_caption))
print(len(gemini_caption))

12
12
12


In [6]:
captions = {'gpt':gpt_caption,'claude':claude_caption,'gemini':gemini_caption}
descriptions = {'gpt':gpt_description,'claude':claude_description,'gemini':gemini_description}

In [7]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd

sentence_bleu_scores = []
for hypo in captions:
  ref = list(set(captions.keys())-{hypo})
  references = [[captions[ref[0]][i],captions[ref[1]][i]] for i in range(12)]
  #print(references)
  hypothesis = captions[hypo]
  corpus_bleu_score = corpus_bleu(references, hypothesis)
  print(f"{hypo} vs ({' and '.join(ref)}) corpus bleu score ",corpus_bleu_score)

  for i in range(12):
    sentence_bleu_score = sentence_bleu(references[i], hypothesis[i])
    sentence_bleu_scores.append({'image':f"c{i+1:>03}","model":hypo,"hypothesis":hypothesis[i],"sentence_bleu_score":sentence_bleu_score})

score_df = pd.DataFrame(sentence_bleu_scores)
score_df.head()


gpt vs (claude and gemini) corpus bleu score  0.3133315244223078
claude vs (gemini and gpt) corpus bleu score  0.24514485529750266
gemini vs (claude and gpt) corpus bleu score  0.21859646087810972


/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


image model                                 hypothesis  sentence_bleu_score
0  c001   gpt           Nested gifts, endless surprises.             0.502648
1  c002   gpt  When a mask hides your smile, use a prop.             0.536334
2  c003   gpt            Public restrooms, but for dogs.             0.191142
3  c004   gpt          The can escapes the opener’s grip             0.229068
4  c005   gpt         Turning microbes into masterpieces             0.260409

# METEOR (Metric for Evaluation of Translation with Explicit ORdering)

In [8]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
tokenize_captions = {'gpt':[],'claude':[],'gemini':[]}
for model in tokenize_captions:
  for caption in captions[model]:
    tokenize_captions[model].append(nltk.word_tokenize(caption))

In [10]:
from nltk.translate.meteor_score import meteor_score

menteor_scores = []
for hypo in captions:
  ref = list(set(captions.keys())-{hypo})
  for i in range(12):
    reference = [tokenize_captions[ref[0]][i],tokenize_captions[ref[1]][i]]
    hypothesis = tokenize_captions[hypo][i]
    menteor_scores.append(meteor_score(reference, hypothesis))

score_df['mentor_score'] = menteor_scores
score_df.sample(5)


image   model                                         hypothesis  \
10  c011     gpt             Building castles while time slips away   
35  c012  gemini     The grass is always greener on the other side.   
16  c005  claude  Bringing science to life, one brushstroke at a...   
23  c012  claude             Sometimes you have to fish for friends   
22  c011  claude            Time's running out faster than expected   

    sentence_bleu_score  mentor_score  
10             0.436037      0.144928  
35             0.161131      0.078125  
16             0.134590      0.054945  
23             0.209459      0.307377  
22             0.160807      0.163934

# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)

In [11]:
!pip install rouge_score

In [13]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge1 = []
rougeL = []
for hypo in captions:
  ref = list(set(captions.keys())-{hypo})

  references = [[captions[ref[0]][i],captions[ref[1]][i]] for i in range(12)]
  hypothesis = captions[hypo]

  for i in range(12):
    rouge1_fmeasure_list = []
    rougeL_fmeasure_list = []
    for r in ref:
      scores = scorer.score(captions[r][i], captions[hypo][i])
      rouge1_fmeasure_list.append(scores['rouge1'].fmeasure)
      rougeL_fmeasure_list.append(scores['rougeL'].fmeasure)
    rouge1.append(sum(rouge1_fmeasure_list)/len(rouge1_fmeasure_list))
    rougeL.append(sum(rougeL_fmeasure_list)/len(rougeL_fmeasure_list))

score_df['average_rouge1_fmeasure'] = rouge1
score_df['average_rougeL_fmeasure'] = rougeL

score_df.sample(5)

image   model                                      hypothesis  \
3   c004     gpt               The can escapes the opener’s grip   
19  c008  claude               The clown car finally broke down.   
6   c007     gpt               Even buses wait for the bus stop.   
25  c002  gemini        Finding the perfect mask for your smile.   
33  c010  gemini  Let me just check my messages while I eat you.   

    sentence_bleu_score  mentor_score  average_rouge1_fmeasure  \
3              0.229068      0.056180                 0.062500   
19             0.353034      0.170455                 0.226190   
6              0.161916      0.051020                 0.000000   
25             0.463425      0.294977                 0.258929   
33             0.146418      0.120482                 0.117647   

    average_rougeL_fmeasure  
3                  0.062500  
19                 0.226190  
6                  0.000000  
25                 0.258929  
33                 0.117647

In [14]:
score_df.to_csv('evaluation.csv',index=False)